# Evaluation Metrics: SCOT

In [1]:
from shapely.ops import cascaded_union
import matplotlib.pyplot as plt
import geopandas as gpd
import multiprocessing
import pandas as pd
import numpy as np
import skimage.io
import tqdm
import glob
import math
import gdal
import time
import sys
import os
import geopandas as gpd
from shapely import wkt

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
import matplotlib
# matplotlib.use('Agg') # non-interactive

import solaris as sol
from solaris.utils.core import _check_gdf_load
from solaris.raster.image import create_multiband_geotiff 

# import from data_postproc_funcs
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
from sn7_baseline_postproc_funcs import map_wrapper, multithread_polys, \
        calculate_iou, track_footprint_identifiers, \
        sn7_convert_geojsons_to_csv
import solaris.eval.scot as scot

/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Midgard/home/hfang

In [2]:
ground_truth = pd.read_csv('/Midgard/home/hfang/temporal_CD/HRNet_SN7_csv/ground_truth.csv')
aois = [z.split('mosaic_')[-1] for z in ground_truth['filename'].values]
ground_truth['aoi'] = aois
timesteps = [z.split('monthly_')[-1].split('_mosaic')[0] for z in ground_truth['filename'].values]
ground_truth['timestep'] = timesteps
ground_truth = ground_truth.drop('filename', 1)

In [3]:
ground_truth.head()

,id,geometry,aoi,timestep
0,1131,"POLYGON ((1022.739556749351 1023.915693833842,...",L15-0387E-1276N_1549_3087_13,2018_01
1,896,"POLYGON ((839.4174400521442 1023.860588716227,...",L15-0387E-1276N_1549_3087_13,2018_01
2,607,"POLYGON ((977.5393254966475 1021.220368818729,...",L15-0387E-1276N_1549_3087_13,2018_01
3,539,"POLYGON ((804.8580670831725 1021.935737615102,...",L15-0387E-1276N_1549_3087_13,2018_01
4,769,"POLYGON ((817.6027463246137 1022.111423592898,...",L15-0387E-1276N_1549_3087_13,2018_01


In [4]:
ground_truth['geometry'] = ground_truth['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(ground_truth, geometry = 'geometry')
print(gdf)

          id                                           geometry  \
0       1131  POLYGON ((1022.740 1023.916, 1021.780 1025.799...   
1        896  POLYGON ((839.417 1023.861, 832.526 1024.057, ...   
2        607  POLYGON ((977.539 1021.220, 976.830 1024.765, ...   
3        539  POLYGON ((804.858 1021.936, 801.562 1024.604, ...   
4        769  POLYGON ((817.603 1022.111, 814.501 1024.105, ...   
...      ...                                                ...   
953426   199  POLYGON ((16.975 27.234, 23.288 27.566, 23.158...   
953427   215  POLYGON ((264.036 7.059, 267.359 7.059, 267.35...   
953428   225  POLYGON ((269.021 3.875, 273.175 3.875, 273.17...   
953429   648  POLYGON ((267.221 3.459, 267.221 1.659, 269.15...   
953430   625  POLYGON ((268.467 0.829, 268.467 -0.833, 271.7...   

                                 aoi timestep  
0       L15-0387E-1276N_1549_3087_13  2018_01  
1       L15-0387E-1276N_1549_3087_13  2018_01  
2       L15-0387E-1276N_1549_3087_13  2018_01  
3  

In [5]:
pred = pd.read_csv('/Midgard/home/hfang/temporal_CD/HRNet_SN7_csv/submission_hrnet_finetune2.csv')
aois = [z.split('mosaic_')[-1] for z in pred['filename'].values]
pred['aoi'] = aois
timesteps = [z.split('monthly_')[-1].split('_mosaic')[0] for z in pred['filename'].values]
pred['timestep'] = timesteps
pred = pred.drop('filename', 1)

In [6]:
pred.head()

,id,geometry,aoi,timestep
0,0,"POLYGON (( 3 37.3,3 37.7,2.7 37.7,2.7 38.3,2.3...",L15-0632E-0892N_2528_4620_13,2018_01
1,0,"POLYGON (( 3 37.3,3 37.7,2.7 37.7,2.7 38.3,2.3...",L15-0632E-0892N_2528_4620_13,2018_02
2,0,"POLYGON (( 3 37.3,3 37.7,2.7 37.7,2.7 38.3,2.3...",L15-0632E-0892N_2528_4620_13,2018_03
3,0,"POLYGON (( 3 37.3,3 37.7,2.7 37.7,2.7 38.3,2.3...",L15-0632E-0892N_2528_4620_13,2018_04
4,0,"POLYGON (( 3 37.3,3 37.7,2.7 37.7,2.7 38.3,2.3...",L15-0632E-0892N_2528_4620_13,2018_05


In [7]:
pred['geometry'] = pred['geometry'].apply(wkt.loads)
pdf = gpd.GeoDataFrame(pred, geometry = 'geometry')
print(pdf)

          id                                           geometry  \
0          0  POLYGON ((3.000 37.300, 3.000 37.700, 2.700 37...   
1          0  POLYGON ((3.000 37.300, 3.000 37.700, 2.700 37...   
2          0  POLYGON ((3.000 37.300, 3.000 37.700, 2.700 37...   
3          0  POLYGON ((3.000 37.300, 3.000 37.700, 2.700 37...   
4          0  POLYGON ((3.000 37.300, 3.000 37.700, 2.700 37...   
...      ...                                                ...   
663765  2363  POLYGON ((924.300 1022.000, 924.300 1022.300, ...   
663766  2363  POLYGON ((924.300 1022.000, 924.300 1022.300, ...   
663767  2363  POLYGON ((924.300 1022.000, 924.300 1022.300, ...   
663768  2363  POLYGON ((924.300 1022.000, 924.300 1022.300, ...   
663769  2363  POLYGON ((924.300 1022.000, 924.300 1022.300, ...   

                                 aoi timestep  
0       L15-0632E-0892N_2528_4620_13  2018_01  
1       L15-0632E-0892N_2528_4620_13  2018_02  
2       L15-0632E-0892N_2528_4620_13  2018_03  
3  

In [8]:
scot.scot_multi_aoi(gdf, pdf, verbose=True)


1 / 10: AOI L15-0387E-1276N_1549_3087_13
Matching footprints
   1: F1 = 0.4356
   2: F1 = 0.4655
   3: F1 = 0.4692
   4: F1 = 0.4712
   5: F1 = 0.4678
   6: F1 = 0.4680
   7: F1 = 0.4670
   8: F1 = 0.4685
   9: F1 = 0.4705
  10: F1 = 0.4702
  11: F1 = 0.4589
  12: F1 = 0.4544
  13: F1 = 0.4686
  14: F1 = 0.4711
  15: F1 = 0.4701
  16: F1 = 0.4695
  17: F1 = 0.4705
  18: F1 = 0.4713
  19: F1 = 0.4709
  20: F1 = 0.4604
  21: F1 = 0.4565
  22: F1 = 0.4465
Identifying mismatches
Tracking:
    Mismatches: 4
      True Pos: 14004
     False Pos: 9148
     False Neg: 23081
   Track Score: 0.4650
Change Detection:
      True Pos: 161
     False Pos: 406
     False Neg: 346
  Change Score: 0.2998
Combined Score: 0.4188

2 / 10: AOI L15-0566E-1185N_2265_3451_13
Matching footprints
   1: F1 = 0.3077
   2: F1 = 0.2647
   3: F1 = 0.2609
   4: F1 = 0.2597
   5: F1 = 0.3000
   6: F1 = 0.3182
   7: F1 = 0.3191
   8: F1 = 0.3357
   9: F1 = 0.3553
  10: F1 = 0.5120
  11: F1 = 0.5367
  12: F1 = 0.5383
 

(0.33608995797554664,
 {'L15-0387E-1276N_1549_3087_13': [4,
   14004,
   9148,
   23081,
   0.4649633945913641,
   161,
   406,
   346,
   0.29981378026070765,
   0.4188225034924723],
  'L15-0566E-1185N_2265_3451_13': [6,
   2699,
   1439,
   2490,
   0.5787498659804867,
   148,
   325,
   511,
   0.26148409893992935,
   0.46573264876204],
  'L15-0632E-0892N_2528_4620_13': [5,
   25571,
   32548,
   38269,
   0.4193376462581687,
   36,
   235,
   97,
   0.1782178217821782,
   0.33003379369141406],
  'L15-1015E-1062N_4061_3941_13': [7,
   33828,
   17786,
   23002,
   0.6238796060639593,
   186,
   781,
   492,
   0.22613981762917934,
   0.4615298082887421],
  'L15-1200E-0847N_4802_4803_13': [6,
   102392,
   64995,
   163922,
   0.47217783680461883,
   36,
   227,
   154,
   0.15894039735099338,
   0.33868337576394386],
  'L15-1276E-1107N_5105_3761_13': [2,
   30982,
   16021,
   42168,
   0.5157091375163333,
   73,
   599,
   644,
   0.10511159107271419,
   0.28951925118442223],
  'L1